In [10]:
%run constants.ipynb
%run read_sheet.ipynb
%run clean_functions.ipynb

In [2]:
# Load the arguments
# Change this if you want to run this notebook directly, rather than from the parent NB
with open(ARGUMENTS_FILENAME, "r") as fid:
    arguments = json.load(fid)

In [3]:
df = read_sheet(arguments["sheet_id"], col_range="A:ZZ")

#### General cleaning

In [4]:
df = pivot_and_clean_dates(df)
df = remove_empty_columns(df)
df = clean_drugs(df)
df = sum_hourly_medications(df)
df = add_daily_medications(df)
df = process_menstrual_cycles(df)

#### Custom to how I track metrics

In [11]:
# This metric used to be yes/no
df["bloated stomach_continuous"] = df["bloated stomach"].copy()

numeric_columns = [c for c in df.columns if "pain" in c]

numeric_columns += [
    "steps",
    "bloated stomach_continuous",
]
df = make_metrics_numeric(df, numeric_columns)

metric = "pelvic pain"
df = get_metric_change(df, metric)

df = map_categorical_to_numeric(df)
df = get_oura_groups(df)

In [36]:
df.to_csv(arguments["dataframe_outfile"], index=True)